In [1]:
import gdown

In [2]:
# url of the data in gdrive
url = "https://drive.google.com/file/d/1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR/view?usp=sharing"

In [9]:
# Getting the unique ID from the url
file_id = url.split('/')[-2]
file_id

'1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR'

In [10]:
# Common for the all and passing the file id at the end
prefix = 'https://drive.google.com/uc?/export=download&id='

In [11]:
gdown.download(prefix+file_id, "Data.zip")

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR
From (redirected): https://drive.google.com/uc?/export=download&id=1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR&confirm=t&uuid=41a49d98-d0e5-4489-90ca-d1572743302c
To: d:\Kidney-Disease-Classification\notebooks\Data.zip
100%|██████████| 1.63G/1.63G [00:52<00:00, 31.2MB/s]


'Data.zip'